<H2>Predicting house prices with Linear Regression Model</h2>

In this project we will work with the Ames data set https://s3.amazonaws.com/dq-content/307/data_description.txt

In [385]:
import pandas as pd
pd.options.display.max_columns = 999
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold

import seaborn as sns

%matplotlib inline

In [386]:
raw = pd.read_csv('AmesHousing.tsv', delimiter='\t')
raw.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [387]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 82 columns):
Order              2930 non-null int64
PID                2930 non-null int64
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         29

<h3>Data Cleaning</h3>

First we will remove columns that are not relevant for our model or leak data regarding the target columns of SalesPrice

In [388]:
## Drop columns that aren't useful for ML
raw = raw.drop(["PID", "Order"], axis=1)

## Drop columns that leak info about the final sale
raw = raw.drop(["Mo Sold", "Sale Condition", "Sale Type"], axis=1)

In [389]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 77 columns):
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Frontage       2440 non-null float64
Lot Area           2930 non-null int64
Street             2930 non-null object
Alley              198 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         2930 non-null object
Roof Matl          2930 non-null object
Exterior 1st       

In [390]:
missing_values = raw.isnull().sum()
print(missing_values)

MS SubClass          0
MS Zoning            0
Lot Frontage       490
Lot Area             0
Street               0
Alley             2732
Lot Shape            0
Land Contour         0
Utilities            0
Lot Config           0
Land Slope           0
Neighborhood         0
Condition 1          0
Condition 2          0
Bldg Type            0
House Style          0
Overall Qual         0
Overall Cond         0
Year Built           0
Year Remod/Add       0
Roof Style           0
Roof Matl            0
Exterior 1st         0
Exterior 2nd         0
Mas Vnr Type        23
Mas Vnr Area        23
Exter Qual           0
Exter Cond           0
Foundation           0
Bsmt Qual           80
                  ... 
Bsmt Half Bath       2
Full Bath            0
Half Bath            0
Bedroom AbvGr        0
Kitchen AbvGr        0
Kitchen Qual         0
TotRms AbvGrd        0
Functional           0
Fireplaces           0
Fireplace Qu      1422
Garage Type        157
Garage Yr Blt      159
Garage Fini

We see that there are some columns with many missing values. Lets remove columns that have nulls over 5% from the overall rows


In [391]:
# Filter Series to columns containing >20% missing values
cols_to_remove = missing_values[(missing_values > len(raw)/20)].sort_values()

# Drop those columns from the data frame. Note the use of the .index accessor
raw = raw.drop(cols_to_remove.index, axis=1)

In [392]:
missing_values = raw.isnull().sum()
print(missing_values)

MS SubClass         0
MS Zoning           0
Lot Area            0
Street              0
Lot Shape           0
Land Contour        0
Utilities           0
Lot Config          0
Land Slope          0
Neighborhood        0
Condition 1         0
Condition 2         0
Bldg Type           0
House Style         0
Overall Qual        0
Overall Cond        0
Year Built          0
Year Remod/Add      0
Roof Style          0
Roof Matl           0
Exterior 1st        0
Exterior 2nd        0
Mas Vnr Type       23
Mas Vnr Area       23
Exter Qual          0
Exter Cond          0
Foundation          0
Bsmt Qual          80
Bsmt Cond          80
Bsmt Exposure      83
                   ..
Heating             0
Heating QC          0
Central Air         0
Electrical          1
1st Flr SF          0
2nd Flr SF          0
Low Qual Fin SF     0
Gr Liv Area         0
Bsmt Full Bath      2
Bsmt Half Bath      2
Full Bath           0
Half Bath           0
Bedroom AbvGr       0
Kitchen AbvGr       0
Kitchen Qu

Now we will examine the rest of the columns that have missing values and decide what to do with each of them

In [393]:
raw['Mas Vnr Type'].value_counts()

None       1752
BrkFace     880
Stone       249
BrkCmn       25
CBlock        1
Name: Mas Vnr Type, dtype: int64

we can replace the null values with the most common value since this is a categorical column

In [394]:
raw['Mas Vnr Type'] = raw['Mas Vnr Type'].fillna(raw['Mas Vnr Type'].mode()[0])
raw['Mas Vnr Type'].value_counts(dropna=False)

None       1775
BrkFace     880
Stone       249
BrkCmn       25
CBlock        1
Name: Mas Vnr Type, dtype: int64

Since we put Nonw in the missing values for Mas Vnr Type we should put 0.0 in the Mas Vnr Area which measures the square feet of th Veneer area

In [395]:
raw['Mas Vnr Area'] = raw['Mas Vnr Area'].fillna(raw['Mas Vnr Area'].mode()[0])
raw['Mas Vnr Area'].value_counts(dropna=False)

0.0       1771
120.0       15
176.0       13
200.0       13
216.0       12
180.0       12
144.0       11
108.0       11
72.0        11
16.0        11
340.0       10
210.0        9
128.0        9
80.0         9
196.0        9
178.0        8
132.0        8
302.0        8
40.0         8
256.0        8
170.0        8
260.0        7
50.0         7
320.0        7
252.0        7
240.0        7
44.0         7
186.0        7
164.0        7
300.0        7
          ... 
408.0        1
118.0        1
1170.0       1
283.0        1
87.0         1
634.0        1
375.0        1
754.0        1
323.0        1
522.0        1
443.0        1
39.0         1
816.0        1
1290.0       1
662.0        1
519.0        1
287.0        1
571.0        1
594.0        1
526.0        1
397.0        1
257.0        1
436.0        1
1050.0       1
381.0        1
518.0        1
109.0        1
440.0        1
322.0        1
175.0        1
Name: Mas Vnr Area, Length: 445, dtype: int64

Now lets check the following fields: Bsmt Qual and Bsmt Cond

In [396]:
raw['Bsmt Qual'].value_counts(dropna=False)

TA     1283
Gd     1219
Ex      258
Fa       88
NaN      80
Po        2
Name: Bsmt Qual, dtype: int64

It seems unlikely that all of the houses have basements so my guess is that the ones with no value have no basement and thus should receive the value NA

In [397]:
raw['Bsmt Qual'] = raw['Bsmt Qual'].fillna(value='NA')
raw['Bsmt Qual'].value_counts(dropna=False)

TA    1283
Gd    1219
Ex     258
Fa      88
NA      80
Po       2
Name: Bsmt Qual, dtype: int64

In [398]:
raw['Bsmt Cond'].value_counts(dropna=False)

TA     2616
Gd      122
Fa      104
NaN      80
Po        5
Ex        3
Name: Bsmt Cond, dtype: int64

In [399]:
raw['Bsmt Cond'] = raw['Bsmt Cond'].fillna(value='NA')
raw['Bsmt Cond'].value_counts(dropna=False)

TA    2616
Gd     122
Fa     104
NA      80
Po       5
Ex       3
Name: Bsmt Cond, dtype: int64

In [400]:
raw['Bsmt Exposure'] = raw['Bsmt Exposure'].fillna(value='NA')
raw['Bsmt Exposure'].value_counts(dropna=False)

No    1906
Av     418
Gd     284
Mn     239
NA      83
Name: Bsmt Exposure, dtype: int64

In [401]:
raw['Total Bsmt SF'] = raw['Total Bsmt SF'].fillna(value=0)

In [402]:
raw['Electrical'].value_counts(dropna=False)

SBrkr    2682
FuseA     188
FuseF      50
FuseP       8
Mix         1
NaN         1
Name: Electrical, dtype: int64

In [403]:
raw['Electrical'] = raw['Electrical'].fillna(raw['Electrical'].mode()[0])
raw['Electrical'].value_counts(dropna=False)

SBrkr    2683
FuseA     188
FuseF      50
FuseP       8
Mix         1
Name: Electrical, dtype: int64

In [404]:
raw['Garage Cars'].value_counts(dropna=False)

 2.0    1603
 1.0     778
 3.0     374
 0.0     157
 4.0      16
NaN        1
 5.0       1
Name: Garage Cars, dtype: int64

In [405]:
raw['Garage Cars'] = raw['Garage Cars'].fillna(raw['Garage Cars'].mode()[0])
raw['Garage Cars'].value_counts(dropna=False)

2.0    1604
1.0     778
3.0     374
0.0     157
4.0      16
5.0       1
Name: Garage Cars, dtype: int64

In [406]:
raw['Garage Area'] = raw['Garage Area'].fillna(raw['Garage Area'].mode()[0])
raw['Garage Area'].value_counts(dropna=False)

0.0       158
576.0      97
440.0      96
484.0      76
240.0      69
528.0      65
400.0      58
480.0      54
264.0      51
288.0      50
308.0      48
280.0      30
420.0      29
336.0      29
216.0      23
462.0      23
672.0      23
286.0      21
504.0      21
384.0      21
506.0      21
312.0      20
495.0      17
624.0      17
352.0      17
550.0      17
525.0      17
360.0      16
180.0      16
300.0      16
         ... 
374.0       1
804.0       1
1105.0      1
354.0       1
428.0       1
323.0       1
485.0       1
923.0       1
673.0       1
783.0       1
414.0       1
886.0       1
375.0       1
189.0       1
369.0       1
860.0       1
694.0       1
698.0       1
1200.0      1
708.0       1
1040.0      1
332.0       1
808.0       1
568.0       1
1043.0      1
639.0       1
869.0       1
1390.0      1
317.0       1
899.0       1
Name: Garage Area, Length: 603, dtype: int64

In [407]:
raw['Bsmt Full Bath'] = raw['Bsmt Full Bath'].fillna(raw['Bsmt Full Bath'].mode()[0])
raw['Bsmt Full Bath'].value_counts(dropna=False)

0.0    1709
1.0    1181
2.0      38
3.0       2
Name: Bsmt Full Bath, dtype: int64

In [408]:

raw['Bsmt Half Bath'] = raw['Bsmt Half Bath'].fillna(raw['Bsmt Half Bath'].mode()[0])
raw['Bsmt Half Bath'].value_counts(dropna=False)

0.0    2755
1.0     171
2.0       4
Name: Bsmt Half Bath, dtype: int64

In [409]:
raw['BsmtFin Type 1'] = raw['BsmtFin Type 1'].fillna(raw['BsmtFin Type 1'].mode()[0])
raw['BsmtFin Type 1'].value_counts(dropna=False)

GLQ    939
Unf    851
ALQ    429
Rec    288
BLQ    269
LwQ    154
Name: BsmtFin Type 1, dtype: int64

In [410]:
raw['BsmtFin Type 2'] = raw['BsmtFin Type 2'].fillna(raw['BsmtFin Type 2'].mode()[0])
raw['BsmtFin Type 2'].value_counts(dropna=False)

Unf    2580
Rec     106
LwQ      89
BLQ      68
ALQ      53
GLQ      34
Name: BsmtFin Type 2, dtype: int64

In [411]:
raw['BsmtFin SF 2'] = raw['BsmtFin SF 2'].fillna(raw['BsmtFin SF 2'].mode()[0])
raw['BsmtFin SF 2'].value_counts(dropna=False)

0.0       2579
294.0        5
180.0        5
72.0         3
435.0        3
483.0        3
147.0        3
374.0        3
162.0        3
182.0        3
539.0        3
168.0        3
144.0        3
620.0        2
276.0        2
202.0        2
492.0        2
596.0        2
127.0        2
68.0         2
93.0         2
344.0        2
96.0         2
210.0        2
105.0        2
290.0        2
273.0        2
110.0        2
468.0        2
247.0        2
          ... 
228.0        1
196.0        1
1085.0       1
278.0        1
402.0        1
604.0        1
442.0        1
169.0        1
764.0        1
820.0        1
419.0        1
201.0        1
690.0        1
362.0        1
1029.0       1
1164.0       1
404.0        1
377.0        1
694.0        1
645.0        1
691.0        1
713.0        1
912.0        1
156.0        1
66.0         1
35.0         1
154.0        1
106.0        1
488.0        1
530.0        1
Name: BsmtFin SF 2, Length: 274, dtype: int64

In [412]:
raw['Bsmt Unf SF'] = raw['Bsmt Unf SF'].fillna(raw['Bsmt Unf SF'].mode()[0])
raw['Bsmt Unf SF'].value_counts(dropna=False)

0.0       245
384.0      19
728.0      14
672.0      13
600.0      12
216.0      11
816.0      11
100.0      11
572.0      11
624.0      10
300.0      10
270.0      10
768.0       9
280.0       9
186.0       9
264.0       9
396.0       9
108.0       8
348.0       8
480.0       8
162.0       8
832.0       8
440.0       8
840.0       8
780.0       8
546.0       8
294.0       8
630.0       7
356.0       7
292.0       7
         ... 
1706.0      1
1544.0      1
647.0       1
882.0       1
1768.0      1
646.0       1
1108.0      1
655.0       1
1453.0      1
1527.0      1
893.0       1
1326.0      1
521.0       1
1094.0      1
701.0       1
178.0       1
621.0       1
615.0       1
79.0        1
1073.0      1
563.0       1
535.0       1
1057.0      1
996.0       1
1063.0      1
1380.0      1
439.0       1
227.0       1
303.0       1
1851.0      1
Name: Bsmt Unf SF, Length: 1137, dtype: int64

In [413]:
raw['BsmtFin SF 1'] = raw['BsmtFin SF 1'].fillna(raw['BsmtFin SF 1'].mode()[0])
raw['BsmtFin SF 1'].value_counts(dropna=False)

0.0       931
24.0       27
16.0       14
300.0       9
288.0       8
384.0       8
20.0        8
600.0       8
456.0       7
360.0       7
602.0       7
375.0       7
624.0       7
936.0       7
700.0       7
500.0       7
544.0       6
504.0       6
468.0       6
625.0       6
547.0       6
662.0       6
528.0       6
560.0       6
553.0       6
368.0       6
120.0       6
276.0       6
588.0       6
312.0       6
         ... 
1151.0      1
771.0       1
2085.0      1
1011.0      1
670.0       1
1239.0      1
1330.0      1
4010.0      1
1194.0      1
1172.0      1
615.0       1
1406.0      1
1271.0      1
2188.0      1
937.0       1
1965.0      1
1022.0      1
1231.0      1
1037.0      1
1332.0      1
590.0       1
1341.0      1
373.0       1
518.0       1
862.0       1
914.0       1
1836.0      1
874.0       1
337.0       1
696.0       1
Name: BsmtFin SF 1, Length: 995, dtype: int64

In [414]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2930 entries, 0 to 2929
Data columns (total 66 columns):
MS SubClass        2930 non-null int64
MS Zoning          2930 non-null object
Lot Area           2930 non-null int64
Street             2930 non-null object
Lot Shape          2930 non-null object
Land Contour       2930 non-null object
Utilities          2930 non-null object
Lot Config         2930 non-null object
Land Slope         2930 non-null object
Neighborhood       2930 non-null object
Condition 1        2930 non-null object
Condition 2        2930 non-null object
Bldg Type          2930 non-null object
House Style        2930 non-null object
Overall Qual       2930 non-null int64
Overall Cond       2930 non-null int64
Year Built         2930 non-null int64
Year Remod/Add     2930 non-null int64
Roof Style         2930 non-null object
Roof Matl          2930 non-null object
Exterior 1st       2930 non-null object
Exterior 2nd       2930 non-null object
Mas Vnr Type       

<h3> Feature Engineering </h3>

Now that we don't have any Null values, Lets explore the different fields again nd see if any further chanegs need to be done in order to make them more meaningful to the model. This involves calculating fileds and changing categorical fields to be Boolean (get_dummies())

In [415]:
raw['years_since_remod'] = raw['Yr Sold'] - raw['Year Remod/Add']
raw['house_age'] = raw['Yr Sold'] - raw['Year Built']


In [416]:
raw['years_since_remod'].value_counts(dropna=False)

 0     238
 1     193
 56    112
 4     111
 2     104
 3      97
 5      91
 8      85
 6      83
 57     79
 59     78
 58     76
 7      75
 9      71
 10     66
 11     62
 12     59
 13     57
 31     50
 39     47
 14     46
 60     42
 30     40
 16     38
 36     37
 32     37
 15     37
 44     34
 17     33
 42     33
      ... 
 53     30
 37     30
 52     29
 33     29
 47     28
 43     27
 49     27
 38     27
 29     26
 41     26
 48     26
 46     25
 55     23
 50     23
 28     22
 45     22
 35     22
 19     21
 54     21
 20     18
 18     18
 23     18
 27     18
 26     17
 21     15
 25     14
 22     13
 24     10
-1       2
-2       1
Name: years_since_remod, Length: 63, dtype: int64

In [417]:
raw['house_age'].value_counts(dropna=False)

 1      218
 0      116
 2       90
 4       76
 5       66
 6       66
 3       60
 31      53
 7       51
 9       50
 39      48
 11      47
 51      45
 32      45
 49      44
 8       44
 42      41
 30      41
 10      40
 52      39
 43      39
 44      38
 13      38
 37      38
 53      37
 47      36
 36      36
 50      36
 48      35
 40      33
       ... 
 110      5
 64       5
 100      5
 95       5
 109      5
 107      5
 79       4
 75       3
 103      3
 117      3
 112      2
 114      2
 119      2
 128      2
 118      2
 127      2
 129      2
 135      1
 102      1
 104      1
 120      1
 122      1
 126      1
-1        1
 101      1
 111      1
 113      1
 115      1
 125      1
 136      1
Name: house_age, Length: 128, dtype: int64

We see that in both of the new columns we have a few negative values which we will need to remove

In [418]:
print (raw.index[raw['house_age']<0])

Int64Index([2180], dtype='int64')


In [419]:
print (raw.index[raw['years_since_remod']<0])

Int64Index([1702, 2180, 2181], dtype='int64')


In [420]:
raw = raw.drop([1702,2180,2181], axis=0)

Now we can drop the columns we used for these new fields since we are not going to need them anymore

In [421]:
raw = raw.drop(["Year Built", "Year Remod/Add","Yr Sold"], axis = 1)

Moving on to the string or cstegorical columns let's check how many values they have and how the distribution of the values look like so we will see if it is worth adding them as dummy columns

In [422]:
nominal_features = ["MS SubClass", "MS Zoning", "Street", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air"]

In [423]:
uniqueness_counts = raw[nominal_features].apply(lambda col: len(col.value_counts())).sort_values()
drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
print(drop_nonuniq_cols)

Index(['Exterior 1st', 'MS SubClass', 'Exterior 2nd', 'Neighborhood'], dtype='object')


In [424]:
raw = raw.drop(drop_nonuniq_cols,axis=1)


In [425]:
text_cols = []
for i in nominal_features:
    if i not in drop_nonuniq_cols:
        text_cols.append(i)
print(text_cols)

['MS Zoning', 'Street', 'Land Contour', 'Lot Config', 'Condition 1', 'Condition 2', 'Bldg Type', 'House Style', 'Roof Style', 'Roof Matl', 'Mas Vnr Type', 'Foundation', 'Heating', 'Central Air']


In [426]:
for col in text_cols:
    raw[col] = raw[col].astype('category')

In [427]:
final_df = pd.concat([raw,pd.get_dummies(raw.select_dtypes(include=['category']))
], axis=1)

In [428]:
final_df = final_df.drop(text_cols,axis=1)

In [429]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2927 entries, 0 to 2929
Columns: 128 entries, Lot Area to Central Air_Y
dtypes: float64(9), int64(23), object(15), uint8(81)
memory usage: 1.3+ MB


<h3> Feature Selection </h3>

In [430]:
numerical_columns = raw.select_dtypes(include=['int', 'float'])
abs_corr_coeffs = numerical_columns.corr()['SalePrice'].abs().sort_values()
print(abs_corr_coeffs)

BsmtFin SF 2         0.006127
Misc Val             0.019273
3Ssn Porch           0.032268
Bsmt Half Bath       0.035875
Low Qual Fin SF      0.037629
Pool Area            0.068438
Overall Cond         0.101540
Screen Porch         0.112280
Kitchen AbvGr        0.119760
Enclosed Porch       0.128685
Bedroom AbvGr        0.143916
Bsmt Unf SF          0.182751
Lot Area             0.267520
2nd Flr SF           0.269601
Bsmt Full Bath       0.276258
Half Bath            0.284871
Open Porch SF        0.316262
Wood Deck SF         0.328183
BsmtFin SF 1         0.439284
Fireplaces           0.474831
TotRms AbvGrd        0.498574
Mas Vnr Area         0.506983
years_since_remod    0.534985
Full Bath            0.546118
house_age            0.558979
1st Flr SF           0.635185
Garage Area          0.641425
Total Bsmt SF        0.644012
Garage Cars          0.648361
Gr Liv Area          0.717596
Overall Qual         0.801206
SalePrice            1.000000
Name: SalePrice, dtype: float64


lets take only those above 0.3

In [435]:
features = abs_corr_coeffs.index[abs_corr_coeffs > 0.4]

In [436]:
final_df = final_df[features]

In [433]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        train = df[:1460]
        test = df[1460:]

        lr.fit(train[features], train["SalePrice"])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(test["SalePrice"], predictions)
        rmse = np.sqrt(mse)

        return rmse
    
    if k == 1:
        # Randomize *all* rows (frac=1) from `df` and return
        shuffled_df = df.sample(frac=1, )
        train = df[:1460]
        test = df[1460:]
        
        lr.fit(train[features], train["SalePrice"])
        predictions_one = lr.predict(test[features])        
        
        mse_one = mean_squared_error(test["SalePrice"], predictions_one)
        rmse_one = np.sqrt(mse_one)
        
        lr.fit(test[features], test["SalePrice"])
        predictions_two = lr.predict(train[features])        
       
        mse_two = mean_squared_error(train["SalePrice"], predictions_two)
        rmse_two = np.sqrt(mse_two)
        
        avg_rmse = np.mean([rmse_one, rmse_two])
        print(rmse_one)
        print(rmse_two)
        return avg_rmse
    else:
        kf = KFold(n_splits=k, shuffle=True)
        rmse_values = []
        for train_index, test_index, in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features], train["SalePrice"])
            predictions = lr.predict(test[features])
            mse = mean_squared_error(test["SalePrice"], predictions)
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return avg_rmse


In [441]:
rmse = train_and_test(final_df, k=3)

[30855.450788233753, 37656.060953613625, 31249.76672516453]
